In [1]:
%load_ext autoreload
%autoreload 2

# Init

In [2]:
import sys
sys.path.insert(0, "../src")

import json
import random
from pathlib import Path
from collections import Counter

from gen.util import read_data, write_jsonl
from feverise import scifact, climatefever, constants

In [3]:
feverp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/data/fever")
climatefp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/data/climatefever")
scifactp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/data/scifact")

climatefdp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-climatefever")
scifactdp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-scifact")

climatefdp.mkdir(exist_ok=True), scifactdp.mkdir(exist_ok=True)

(None, None)

In [16]:
fever_dev_sample, wiki_sample = None, None

fever_dev_sample = read_data(feverp / "train.jsonl")
for p in feverp.joinpath("wiki-pages").iterdir():
    if random.uniform(0, 1) > 0.7:
        wiki_sample = read_data(p)
        break

In [19]:
fever_dev_sample[199]

{'id': 12113,
 'verifiable': 'VERIFIABLE',
 'label': 'SUPPORTS',
 'claim': 'Canada possesses a constitution.',
 'evidence': [[[27868, 34157, 'Canada', 18],
   [27868, 34157, 'Constitution_Act,_1982', 0]],
  [[27868, 34158, 'Canada', 21], [27868, 34158, 'Constitutional_monarchy', 0]],
  [[30847, 37490, 'Canada', 17]],
  [[32940, 40022, 'Canada', 17]],
  [[301708, 292813, 'Canada', 17]],
  [[301708, 292814, 'Canada', 21]],
  [[301708, 292815, 'Canada', 18]],
  [[303045, 294023, 'Canada', 17]],
  [[303045, 294024, 'Canada', 18]],
  [[303045, 294025, 'Canada', 21]],
  [[304413, 295213, 'Canada', 17]],
  [[304413, 295214, 'Canada', 18]],
  [[304413, 295215, 'Canada', 21]]]}

In [18]:
wiki_sample[1999]

{'id': 'A_Rose_Without_a_Thorn',
 'text': "A Rose without a Thorn is a 1958 Australian TV play about King Henry VIII 's marriage to Katherine Howard . It was directed by Alan Burke from a play by Clifford Bax . The play was shown live in Sydney , recorded , then shown in Melbourne .   It was the first production directed by Alan Burke after he joined the ABC full time . Burke would go on to be one of the leading directors of the early days of Australian television . Seven different sets were used in the program . ",
 'lines': "0\tA Rose without a Thorn is a 1958 Australian TV play about King Henry VIII 's marriage to Katherine Howard .\tKing Henry VIII\tKing Henry VIII\n1\tIt was directed by Alan Burke from a play by Clifford Bax .\tClifford Bax\tClifford Bax\n2\tThe play was shown live in Sydney , recorded , then shown in Melbourne .\n3\t\n4\t\n5\tIt was the first production directed by Alan Burke after he joined the ABC full time .\n6\tBurke would go on to be one of the leading direc

In [37]:
for f in fever_dev_sample:
    if "-LRB-" in f["claim"]  or "(" in f["claim"]:
        print(f)
        break

{'id': 189815, 'verifiable': 'VERIFIABLE', 'label': 'SUPPORTS', 'claim': 'The Smurfs (film) was released.', 'evidence': [[[221732, 227296, 'The_Smurfs_-LRB-film-RRB-', 10]], [[221747, 227307, 'The_Smurfs_-LRB-film-RRB-', 0]], [[221747, 227308, 'The_Smurfs_-LRB-film-RRB-', 10]], [[221747, 227309, 'The_Smurfs_-LRB-film-RRB-', 15]], [[221747, 227310, 'The_Smurfs_-LRB-film-RRB-', 12]]]}


In [40]:
for f in wiki_sample:
    if "(" in f["text"]:
        print(f)
        break

{'id': '85th_Street_-LRB-Manhattan-RRB-', 'text': '85th Street is a westbound-running street , running from East End Avenue to Riverside Drive in the borough of Manhattan in New York City .   The street provides access to the 85th/86th Street Transverse , which runs east -- west through Central Park , heading from the Upper East Side -LRB- where it is known as East 85th Street -RRB- to the Upper West Side -LRB- where it is known as West 85th Street -RRB- . It includes landmarks such as the Lewis Gouverneur and Nathalie Bailey Morris House at 100 East 85th Street , the sidewalk clock at East 85th Street and Third Avenue , the Yorkville Bank Building at 201 -- 203 East 85th Street , Red House at 350 West 85th Street , and Regis High School . Regis High School (New York City) ', 'lines': '0\t85th Street is a westbound-running street , running from East End Avenue to Riverside Drive in the borough of Manhattan in New York City .\tManhattan\tManhattan\tRiverside Drive\tRiverside Drive (Manh

# SciFact

Check if every claim has only 1 type of label for all sentences

In [7]:
sf_dev = read_data(scifactp / "claims_dev.jsonl")
sf_test = read_data(scifactp / "claims_test.jsonl")
sf_train = read_data(scifactp / "claims_train.jsonl")
sf_corpus = read_data(scifactp / "corpus.jsonl")

# Climate-FEVER

- Has multiple labels for each sentence for each claim

In [5]:
cf_data = read_data(climatefp / "climatefever.jsonl")